In [1]:
#imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import urllib.parse
from glob import glob
import os
import ntpath 
import zipfile

#paths
#PPMI_zippath='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018.zip'
PPMI_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/'

In [2]:
def reading_csv_files(folder_path):
    # saving all the csv files in a dataframe
    import os
    import glob
    # glob.glob('*.csv') #find all the csv files in a pathname. 
    os.chdir(folder_path)
    csv_files = [i for i in glob.glob('*.csv')]

    # Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
    dict_files={}
    files_names=[]
    for files in csv_files:
    #for i, files in enumerate(csv_files): 
        df=pd.read_csv(files,engine='python')
        basepath, filename=ntpath.split(files)
        files_names.append(filename)
        dict_files[filename]=df

    # Calculating the # of rows and features for each dataframe and storing them in a list of tuples
    shape_df=[]
    for i in range(0,len(dict_files.keys())):
        nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
        shape_df.append((nrows,ncols))

    return (shape_df,files_names,dict_files)

In [3]:
def number_of_patients(df):
    '''return the number of patients in each csv files'''
    patients=len(list(set(df['PATNO'])))
    if 'EVENT_ID' in df.columns:
        events=len(list(set(df['EVENT_ID'])))
    else:
        events='no TS data'
    return (patients,events)

In [23]:
def features_selection(df, SEL):
    '''cleanier way to do features selection in each csv file'''
    return df_sel

Inline-style: 
![1-Subject-Characteristics]()
![2-Biospecimen]()
![3-Enrollment]()
![4-Imaging]()
![5-Medical-History]()
![6-Motor-Assessments]()
![7-Non-Motor-Assessments]()

## 1-Subject-Characteristics

In [5]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Screening__Demographics - useful
#### NOT TS DATA

In [6]:
SCREEN=dict_files[files_names[3]]
#SCREEN.head()
#list(set(SCREEN['EVENT_ID']))
#SCREEN.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME',
       'CONSNTDT', 'APPRDX', 'CURRENT_APPRDX', 'P3GRP', 'BIRTHDT', 'GENDER',
       'HISPLAT', 'RAINDALS', 'RAASIAN', 'RABLACK', 'RAHAWOPI', 'RAWHITE',
       'RANOS', 'PRJENRDT']
SCREEN_sel=SCREEN[SEL]
#list(set(SCREEN_sel['EVENT_ID']))
print(number_of_patients(SCREEN))
SCREEN_sel.head()

(2144, 1)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,CONSNTDT,APPRDX,CURRENT_APPRDX,P3GRP,BIRTHDT,GENDER,HISPLAT,RAINDALS,RAASIAN,RABLACK,RAHAWOPI,RAWHITE,RANOS,PRJENRDT
0,224392801,3400,CONSENT,SCREEN,06/2010,1.0,1.0,NaN,1971.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,07/2010
1,224394301,3401,CONSENT,SCREEN,06/2010,2.0,2.0,NaN,1954.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,07/2010
2,224398401,3402,CONSENT,SCREEN,06/2010,3.0,3.0,NaN,1964.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10/2011
3,224400201,3403,CONSENT,SCREEN,06/2010,1.0,1.0,NaN,1941.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,07/2010
4,224718101,3404,CONSENT,SCREEN,06/2010,2.0,2.0,NaN,1954.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,07/2010


## 2-Biospecimen

In [7]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/2-Biospecimen/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Genetic_Results_Tests - useful: Feat-bio - 1116 patients
#### NOT TS DATA

In [8]:
MUTRSLT=dict_files[files_names[3]]
#MUTRSLT.head()
#MUTRSLT.columns

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'GENECAT', 'LRRKCD', 'MUTRSLT']
MUTRSLT_sel=MUTRSLT[SEL]
print(number_of_patients(MUTRSLT))
MUTRSLT_sel.head()
#list(set(MUTRSLT_sel['EVENT_ID']))

(1116, 1)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,GENECAT,LRRKCD,MUTRSLT
0,426136501,41386,GMU,MUTRSLT,01/2014,1,1.0,1
1,445446201,50157,GMU,MUTRSLT,06/2014,1,1.0,1
2,422444501,40819,GMU,MUTRSLT,01/2014,1,1.0,1
3,423319501,40577,GMU,MUTRSLT,01/2014,1,1.0,1
4,493347201,41564,GMU,MUTRSLT,01/2015,1,1.0,1


### Pilot_Biospecimen_Analysis_Results - useful: Feat-bio
#### Understand if the cohort of patient IDs are different from the classic PPMI cohort.
#### NOT TS DATA.

In [9]:
### Pilot_Biospecimen_Analysis_Results - useful: Feat-bio
Biospecimen_Analysis=dict_files[files_names[6]]
#Biospecimen_Analysis.head()
#len(list(set(PILOTRSLT['PATNO'])))
#Biospecimen_Analysis.columns

SEL=['PATNO', 'DIAGNOSIS', 'CLINICAL_EVENT', 'TYPE', 'TESTNAME',
       'TESTVALUE','RUNDATE', 'PROJECTID']
print(number_of_patients(Biospecimen_Analysis))
Biospecimen_Analysis_sel=Biospecimen_Analysis[SEL]
Biospecimen_Analysis_sel.head()
#list(set(Biospecimen_Analysis_sel['DIAGNOSIS']))

(664, 'no TS data')


,PATNO,DIAGNOSIS,CLINICAL_EVENT,TYPE,TESTNAME,TESTVALUE,RUNDATE,PROJECTID
0,3400,PD,BL,Cerebrospinal Fluid,Abeta 42,207,2011-07-21,101
1,3400,PD,BL,Cerebrospinal Fluid,p-Tau181P,14,2011-07-21,101
2,3400,PD,BL,Cerebrospinal Fluid,Total tau,10,2011-07-21,101
3,3400,PD,ST,Cerebrospinal Fluid,Abeta 42,231,2011-07-15,101
4,3400,PD,ST,Cerebrospinal Fluid,p-Tau181P,21,2011-07-15,101


about Cerobrospinal fluid: https://en.wikipedia.org/wiki/Cerebrospinal_fluid

### Lumbar_Puncture_Sample_Collection - useful: Feat-bio - 1344 patients

In [10]:
LUMBAR=dict_files[files_names[9]]
#LUMBAR.head()
#LUMBAR.columns

### Only features useful I think: white cell blood count, red blood cell count, total glucose, total protein.

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'FASTSTAT','PDMEDYN', 'PDMEDDT', 'PDMEDTM',
     'SMPDSCRD','RBCRSLT','WBCRSLT','TOPRRSLT','TGLCRSLT','SPFI','FLUORO']
print(number_of_patients(LUMBAR))
LUMBAR_sel=LUMBAR[SEL]
LUMBAR_sel.head()
#list(set(LUMBAR_sel['EVENT_ID']))

(1344, 20)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,FASTSTAT,PDMEDYN,PDMEDDT,PDMEDTM,SMPDSCRD,RBCRSLT,WBCRSLT,TOPRRSLT,TGLCRSLT,SPFI,FLUORO
0,232556301,3401,BL,LUMBAR,07/2010,NaN,0.0,NaN,NaN,0,0.0,1.0,20.0,63.0,0.0,0.0
1,232568401,3403,BL,LUMBAR,07/2010,NaN,0.0,NaN,NaN,0,0.0,1.0,19.0,56.0,0.0,0.0
2,233649201,3400,BL,LUMBAR,07/2010,NaN,0.0,NaN,NaN,0,76.0,1.0,75.0,55.0,0.0,0.0
3,235052301,3404,BL,LUMBAR,07/2010,NaN,0.0,NaN,NaN,0,1.0,2.0,38.0,59.0,0.0,0.0
4,236619201,3405,BL,LUMBAR,07/2010,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3-Enrollment - useless?

In [11]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/3-Enrollment/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'Primary_Diagnosis.csv' - useful: Feat-Gen - 789 patients... (Control Group not included?)

In [12]:
### 'Primary_Diagnosis.csv' - useful: Feat-Gen
PRIMDXPD=dict_files[files_names[8]]
#PRIMDXPD.head()
#PRIMDXPD.columns

SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PRIMDIAG', 'OTHNEURO']
print(number_of_patients(PRIMDXPD))
PRIMDXPD_sel=PRIMDXPD[SEL]
PRIMDXPD_sel.head()
#list(set(PRIMDXPD['EVENT_ID']))

(789, 15)


,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PRIMDIAG,OTHNEURO
0,226859501,V,3401,SC,PRIMDXPD,06/2010,17.0,NaN
1,232502501,V,3405,SC,PRIMDXPD,07/2010,17.0,NaN
2,226902401,V,3403,SC,PRIMDXPD,06/2010,1.0,NaN
3,232739901,V,3404,SC,PRIMDXPD,06/2010,17.0,NaN
4,232697801,V,3406,SC,PRIMDXPD,07/2010,1.0,NaN


### 5-Medical History

In [13]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'Diagnostic_Features.csv': useful: Feat-PD - 1670 patients.

In [15]:
DIAGFEAT=dict_files[files_names[1]]
DIAGFEAT.head()
#DIAGFEAT.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'DFSTROKE', 'DFRSKFCT', 'DFPRESNT', 'DFRPROG', 'DFSTATIC', 'DFHEMPRK',
       'DFAGESX', 'DFOTHCRS', 'DFCRSCM', 'DFRTREMP', 'DFRTREMA', 'DFPATREM',
       'DFOTHTRM', 'DFTREMCM', 'DFRIGIDP', 'DFRIGIDA', 'DFAXRIG', 'DFUNIRIG',
       'DFTONE', 'DFOTHRIG', 'DFRIGCM', 'DFBRADYP', 'DFBRADYA', 'DFAKINES',
       'DFBRPLUS', 'DFOTHABR', 'DFABRCM', 'DFPGDIST', 'DFGAIT', 'DFFREEZ',
       'DFFALLS', 'DFOTHPG', 'DFPGCM', 'DFPSYCH', 'DFCOGNIT', 'DFDYSTON',
       'DFCHOREA', 'DFMYOCLO', 'DFOTHHYP', 'DFHYPCM', 'DFHEMTRO', 'DFPSHYPO',
       'DFSEXDYS', 'DFURDYS', 'DFBWLDYS', 'DFOCULO', 'DFEYELID', 'DFNEURAB',
       'DFDOPRSP', 'DFRAPSPE', 'DFBULBAR', 'DFCTSCAN', 'DFMRI', 'DFATYP']
print(number_of_patients(DIAGFEAT))
DIAGFEAT_sel=DIAGFEAT[SEL]
DIAGFEAT_sel.head()
#list(set(DIAGFEAT_sel['EVENT_ID']))

(1670, 19)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,DFSTROKE,DFRSKFCT,DFPRESNT,DFRPROG,DFSTATIC,...,DFBWLDYS,DFOCULO,DFEYELID,DFNEURAB,DFDOPRSP,DFRAPSPE,DFBULBAR,DFCTSCAN,DFMRI,DFATYP
0,358812401,3428,V04,DIAGFEAT,08/2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0
1,364251801,3433,V04,DIAGFEAT,09/2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,N,0.0,0.0,0,0,0.0
2,366180201,3166,V04,DIAGFEAT,10/2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,N,0.0,0.0,N,0,0.0
3,369126801,3162,V04,DIAGFEAT,10/2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,N,0.0,0.0,N,N,0.0
4,362799201,3432,ST,DIAGFEAT,08/2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,N,0.0,0.0,0,0,0.0


### 'General_Medical_History.csv': useful: Feat-Gen - 2067 patients
#### NOT TS DATA...

In [17]:
MHXGEN=dict_files[files_names[3]]
#MHXGEN.head()
#MHXGEN.columns

SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MHCAT', 'MHROW', 'MHHX', 'MHACTRES', 'MHDIAGYR', 
     'PT_CODE', 'PT_NAME', 'HLT_NAME', 'SOCABBR1']
# I need probably to select less features here
print(number_of_patients(MHXGEN))
MHXGEN_sel=MHXGEN[SEL]
MHXGEN_sel.head()
#list(set(MHXGEN['EVENT_ID']))

(2067, 2)


,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,MHCAT,MHROW,MHHX,MHACTRES,MHDIAGYR,PT_CODE,PT_NAME,HLT_NAME,SOCABBR1
0,226698901,V,3400,SC,MHXGEN,06/2010,1f. ENT,1,1.0,1.0,1987.0,10015090.0,Epistaxis,Nasal disorders NEC,Resp
1,226700701,V,3400,SC,MHXGEN,06/2010,1l. Gynecologic/Urologic,1,1.0,1.0,2005.0,10013935.0,Dysmenorrhoea,Menstruation and uterine bleeding NEC,Repro
2,226701901,V,3400,SC,MHXGEN,06/2010,1p. Neurologic (other than disease under study),1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,226702601,V,3400,SC,MHXGEN,06/2010,1r. Allergy / Immunologic (Note drug allergies),2,1.0,1.0,1981.0,10020751.0,Hypersensitivity,Allergic conditions NEC,Immun
4,226870201,V,3401,SC,MHXGEN,06/2010,1o. Hemato/lymphatic,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN


### General_Physical_Exam: useful: Feat-Gen - 2047 patients

In [18]:
GENPHYEX=dict_files[files_names[5]]
#GENPHYEX.head()
#GENPHYEX.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PESEQ', 'PECAT', 'ABNORM']
print(number_of_patients(GENPHYEX))
GENPHYEX_sel=GENPHYEX[SEL]
GENPHYEX_sel.head()
#list(set(GENPHYEX['EVENT_ID']))

(2047, 8)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,PESEQ,PECAT,ABNORM
0,226890001,3402,SC,GENPHYEX,06/2010,3,Eyes,0.0
1,226890301,3402,SC,GENPHYEX,06/2010,6,Cardiovascular (including peripheral vascular),0.0
2,226890601,3402,SC,GENPHYEX,06/2010,9,Neurological,0.0
3,232699801,3406,SC,GENPHYEX,07/2010,8,Musculoskeletal,0.0
4,232700101,3406,SC,GENPHYEX,07/2010,11,Other (Specify location and describe),0.0


### 'General_Neurological_Exam.csv': useful: Feat-Gen - 2048 patients

In [22]:
PENEURO=dict_files[files_names[7]]
#PENEURO.head()
#PENEURO.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MSRARSP', 'MSRACM', 'MSLARSP', 'MSLACM', 'MSRLRSP', 'MSRLCM',
       'MSLLRSP', 'MSLLCM', 'COFNRRSP', 'COFNRCM', 'COFNLRSP', 'COFNLCM',
       'COHSRRSP', 'COHSRCM', 'COHSLRSP', 'COHSLCM', 'SENRARSP', 'SENRACM',
       'SENLARSP', 'SENLACM', 'SENRLRSP', 'SENRLCM', 'SENLLRSP', 'SENLLCM',
       'RFLRARSP', 'RFLRACM', 'RFLLARSP', 'RFLLACM', 'RFLRLRSP', 'RFLRLCM',
       'RFLLLRSP', 'RFLLLCM', 'PLRRRSP', 'PLRRCM', 'PLRLRSP', 'PLRLCM']
print(number_of_patients(PENEURO))
PENEURO_sel=PENEURO[SEL]
PENEURO_sel.head()
#list(set(PENEURO['EVENT_ID']))

(2048, 15)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,MSRARSP,MSRACM,MSLARSP,MSLACM,MSRLRSP,...,RFLLARSP,RFLLACM,RFLRLRSP,RFLRLCM,RFLLLRSP,RFLLLCM,PLRRRSP,PLRRCM,PLRLRSP,PLRLCM
0,232741801,3404,SC,PENEURO,06/2010,0,NaN,0,NaN,0,...,1.0,NaN,1.0,NaN,1.0,NaN,0.0,NaN,0.0,NaN
1,232698601,3406,SC,PENEURO,07/2010,0,NaN,0,NaN,0,...,2.0,NaN,2.0,NaN,2.0,NaN,0.0,NaN,0.0,NaN
2,226889601,3402,SC,PENEURO,06/2010,0,NaN,0,NaN,0,...,2.0,NaN,2.0,NaN,2.0,NaN,0.0,NaN,0.0,NaN
3,226914401,3403,SC,PENEURO,06/2010,0,NaN,0,NaN,1,...,3.0,NaN,3.0,NaN,3.0,NaN,0.0,NaN,0.0,NaN
4,226840201,3400,SC,PENEURO,06/2010,0,NaN,0,NaN,0,...,2.0,NaN,2.0,NaN,2.0,NaN,0.0,NaN,1.0,NaN


### 'Neurological_Exam_-_Cranial_Nerves.csv': useful: Feat-PD - 2049 patients

In [ ]:
PENEURO2=dict_files[files_names[13]]
PENEURO2.head()
PENEURO2.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'CN1RSP','CN2RSP', 'CN346RSP','CN5RSP',
     'CN7RSP', 'CN8RSP', 'CN910RSP',
       'CN11RSP', 'CN12RSP']
print(number_of_patients(PENEURO2))
PENEURO2_sel=PENEURO2[SEL]
PENEURO2_sel.head()
#list(set(PENEURO['EVENT_ID']))

### 'PD_Features.csv': useful: Feat-PD - caution: these are actually not timeseries but examination only done at screening (SC), BL & V04
### 1017 patients

In [ ]:
PDFEAT=dict_files[files_names[14]]
#PDFEAT.head()
#PDFEAT.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'SXMO',
       'SXYEAR', 'PDDXDT', 'PDDXEST', 'DXTREMOR', 'DXRIGID', 'DXBRADY',
       'DXPOSINS', 'DXOTHSX', 'DXOTHCM', 'DOMSIDE']
print(number_of_patients(PDFEAT))
PDFEAT_sel=PDFEAT[SEL]
PDFEAT_sel.head()
#list(set(PDFEAT_sel['DXOTHCM'])) - see if we remove the comment feature or not.
#list(set(PDFEAT['EVENT_ID']))

### 'Vital_Signs.csv': useful: Feat-Gen
### 2047 patients

In [ ]:
VITAL=dict_files[files_names[15]]
#VITAL.head()
#VITAL.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'WGTKG', 'HTCM', 'TEMPC', 'SYSSUP', 'DIASUP', 'HRSUP',
       'SYSSTND', 'DIASTND', 'HRSTND']
print(number_of_patients(VITAL))
VITAL_sel=VITAL[SEL]
VITAL_sel.head()

## 6-Motor-Assessments - all of them can be useful. 
#### But we might make the choice to not take all of them. 
#### For assessing the disease state (the labels), I think we should focus on the UPDRS1,3,&4 and take the other assessments as features

In [ ]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/6-Motor-Assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'MDS_UPDRS_Part_I__Patient_Questionnaire.csv' - 2011 patients

In [ ]:
NUPDR1SP=dict_files[files_names[0]]
print(number_of_patients(NUPDR1SP))
NUPDR1SP.head()

### 'MDS_UPDRS_Part_I.csv' - 2010 patients

In [ ]:
NUPDRS1=dict_files[files_names[2]]
print(number_of_patients(NUPDRS1))
NUPDRS1.head()

### 'MDS_UPDRS_Part_II__Patient_Questionnaire.csv' - 2011 patients

In [ ]:
NUPDRS2P=dict_files[files_names[11]]
print(number_of_patients(NUPDRS2P))
NUPDRS2P.head()

### 'MDS_UPDRS_Part_III.csv' - 2010 patients

In [ ]:
NUPDRS3=dict_files[files_names[7]]
print(number_of_patients(NUPDRS3))
NUPDRS3.head()

### 'MDS_UPDRS_Part_IV.csv' - 860 patients

In [ ]:
NUPDRS4=dict_files[files_names[5]]
print(number_of_patients(NUPDRS4))
NUPDRS4.head()

### 'PASE_-_Household_Activity.csv' - 1248 patients

In [ ]:
PASE=dict_files[files_names[3]]
print(number_of_patients(PASE))
PASE.head()

### 'PASE_-_Leisure_Time_Activity.csv' - 1248 patients

In [ ]:
PASELT=dict_files[files_names[10]]
print(number_of_patients(PASELT))
PASELT.head()

### 'Modified_Schwab_+_England_ADL.csv' - 1837 patients

In [ ]:
MODSEADL=dict_files[files_names[6]]
print(number_of_patients(MODSEADL))
MODSEADL.head()

### 'TAP-PD_OPDM_Assessment.csv' - only 173 patients...


In [ ]:
TAPOPDM=dict_files[files_names[8]]
print(number_of_patients(TAPOPDM))
TAPOPDM.head()

### 'TAP-PD_OPDM_Use_Questionnaire.csv' - only 32 patients

In [ ]:
TAPUSE=dict_files[files_names[1]]
print(number_of_patients(TAPUSE))
TAPUSE.head()

### 'Gait_Data___Arm_swing.csv' - not sure if we should consider this dataset. - only 62 patients...

In [ ]:
GAITDATA=dict_files[files_names[4]]
print(number_of_patients(GAITDATA))
GAITDATA.head()

### 'TAP-PD_Kinetics_Device_Testing.csv' - Idem for this one- not sure it is essential to consider it.
#### NO TS data. 
### Only 32 patients...

In [ ]:
KINETICS=dict_files[files_names[9]]
print(number_of_patients(KINETICS))
KINETICS.head()

## 7-Non-motor-assessments - all of them can be useful but we might need to choose. 
### Do a correlation analysis and ask doctors which ones are the more relevant. 

In [ ]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/7-Non-motor-assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'University_of_Pennsylvania_Smell_ID_Test.csv' - 1799 patients

In [ ]:
UPSIT=dict_files[files_names[0]]
print(number_of_patients(UPSIT))
UPSIT.head()

### 'Features_of_REM_Behavior_Disorder.csv' - only 80 patients...
#### NOT TS data.

In [ ]:
REMBHVDS=dict_files[files_names[1]]
print(number_of_patients(REMBHVDS))
REMBHVDS.head()

### 'Epworth_Sleepiness_Scale.csv - 1343 patients.

In [ ]:
EPWORTH=dict_files[files_names[2]]
print(number_of_patients(EPWORTH))
EPWORTH.head()

### 'Letter_-_Number_Sequencing__PD_.csv' - 1341 patients.

In [ ]:
LNSPD=dict_files[files_names[3]]
print(number_of_patients(LNSPD))
LNSPD.head()

### 'Olfactory_UPSIT.csv' - 4468 patients
#### NB: only take the 2 last columns as features (TOTAL_CORRECT, UPSIIT_PRCNTGE)

In [ ]:
OLFACT=dict_files[files_names[4]]
#print(number_of_patients(OLFACT))
print(len(list(set(OLFACT['SUBJECT_ID']))))
OLFACT.head()

### 'REM_Sleep_Disorder_Questionnaire.csv' - 1810 patients

In [ ]:
REMSLEEP=dict_files[files_names[5]]
print(number_of_patients(REMSLEEP))
REMSLEEP.head()

### 'Cognitive_Assessments.csv' - 1172 patients

In [ ]:
COGTIME=dict_files[files_names[6]]
print(number_of_patients(COGTIME))
COGTIME.head()

### 'SCOPA-AUT.csv' - 1345 patients

In [ ]:
SCOPAAUT=dict_files[files_names[7]]
print(number_of_patients(SCOPAAUT))
SCOPAAUT.head()

### Cognitive_Categorization.csv' - 1302 patients

In [ ]:
COGCATG=dict_files[files_names[8]]
print(number_of_patients(COGCATG))
COGCATG.head()

### 'Hopkins_Verbal_Learning_Test.csv' - 1341 patients

In [ ]:
HVLT=dict_files[files_names[9]]
print(number_of_patients(HVLT))
HVLT.head()

### 'Montreal_Cognitive_Assessment__MoCA_.csv' - 2054 patients

In [ ]:
MOCA=dict_files[files_names[10]]
print(number_of_patients(MOCA))
MOCA.head()

### 'Semantic_Fluency.csv' - 1342 patients

In [ ]:
SFT=dict_files[files_names[11]]
print(number_of_patients(SFT))
SFT.head()

### 'Geriatric_Depression_Scale__Short_.csv' - 1478 patients

In [ ]:
GDSSHORT=dict_files[files_names[12]]
print(number_of_patients(GDSSHORT))
GDSSHORT.head()

### 'Symbol_Digit_Modalities.csv' - 1340 patients

In [ ]:
SDM=dict_files[files_names[13]]
print(number_of_patients(SDM))
SDM.head()

### 'State-Trait_Anxiety_Inventory.csv' - 1474 patients

In [ ]:
STAI=dict_files[files_names[14]]
print(number_of_patients(STAI))
STAI.head()

### 'Benton_Judgment_of_Line_Orientation.csv' - 1341 patients

In [ ]:
BENTONOD=dict_files[files_names[15]]
print(number_of_patients(BENTONOD))
BENTONOD.head()

### 'QUIP_Current_Short.csv' - 1346 patients

In [ ]:
QUIPCS=dict_files[files_names[16]]
print(number_of_patients(QUIPCS))
QUIPCS.head()

# Datasets to investigate

### Blood_Chemistry__Hematology - Contains information about patient blood. 
### Ask doctors if really useful. 

In [ ]:
COVANCE=dict_files[files_names[12]]
#COVANCE.head()

### IUSM_Catalog - DNK - most likely useless
#### seems ti be only metadata on the lab collections.
Look @ the PPMI doc to understand what is this table

In [ ]:
IUSMCAT=dict_files[files_names[14]]
#IUSMCAT.head()

# Additionnal data that will be used later on (imaging data and medication data)

## 4-Imaging - dataset not to be used for the time being.

In [ ]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/4-Imaging/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### FBB_Analysis_Data.csv

In [ ]:
FBBANAL=dict_files[files_names[0]]
#FBBANAL.head()

### AV-133_Image_Metadata.csv

In [ ]:
AV133META=dict_files[files_names[1]]
#AV133META.head()

### AV-133_Imaging.csv

In [ ]:
AVIMAG=dict_files[files_names[2]]
#AVIMAG.head()

### DaTscan_Imaging.csv

In [ ]:
DATSCAN=dict_files[files_names[3]]
#DATSCAN.head()

### SPECT_Scan_Information_Source_Document.csv

In [ ]:
SPECTSCANINFO=dict_files[files_names[4]]
#SPECTSCANINFO.head()

### AV-133_SBR_Results.csv

In [ ]:
AV133SBRRSLT=dict_files[files_names[5]]
#AV133SBRRSLT.head()

### Magnetic_Resonance_Imaging.csv

In [ ]:
MRI=dict_files[files_names[6]]
#MRI.head()

### DTI_Regions_of_Interest

In [ ]:
DTIROI=dict_files[files_names[7]]
#DTIROI.head()

### DaTSCAN_SPECT_Visual_Interpretation_Assessment.csv

In [ ]:
FBBMETA=dict_files[files_names[8]]
#FBBMETA.head()

### 'FBB_Metadata.csv'

In [ ]:
FBBMETA=dict_files[files_names[9]]
#FBBMETA.head()

### 'MRI_Imaging_Data_Transfer_Information_Source_Document.csv'

In [21]:
MRIINFO=dict_files[files_names[10]]
#MRIINFO.head()

## Medications data & misc data - to be used later on.

### Family_History_PD - most likely useful but not to be used for now.
#### Can be useful for detecting early PD

In [ ]:
FAMHXPD=dict_files[files_names[2]]
#FAMHXPD.head()

### 'Prodromal_Diagnostic_Questionnaire.csv': useful: Feat-Gen (PRIMDIAG & PSLVL2)
#### NB: PSLV2 evaluates the proba of the patient being PD: might be useful for early PD. 
#### But not to be used in the main analysis as a feature (only for Prodromal - i.e 800 subjects)

In [20]:
PRODIAG=dict_files[files_names[0]]
#PRODIAG.head()
#PRODIAG.columns

SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'PRIMDIAG',
       'OTHNEURO', 'PSLVL2']
print(number_of_patients(PRODIAG))
PRODIAG_sel=PRODIAG[SEL]
PRODIAG_sel.head()

(800, 16)


,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,PRIMDIAG,OTHNEURO,PSLVL2
0,408770801,V,10362,SC,PRODDIAG,17,NaN,4
1,605736601,V,10405,SC,PRODDIAG,17,NaN,5
2,410987801,V,10496,SC,PRODDIAG,23,NaN,5
3,409586301,S,10606,SC,PRODDIAG,17,NaN,4
4,448326801,V,10662,SC,PRODDIAG,17,NaN,5


### 'Signature_Form.csv' - most likely useful: Meds - but not to be used for now.
#### Can be useful for the meds log

In [ ]:
SIG=dict_files[files_names[10]]
#SIG.head()

### Use of PD Medication: useful: Meds - but not to be used for now.

In [ ]:
PDMEDUSE=dict_files[files_names[4]]
#PDMEDUSE.head()

### 'Concomitant_Medications.csv': useful: Meds - But not to be used for now.

In [ ]:
CMED=dict_files[files_names[6]]
#CMED.head()

### 'Current_Medical_Conditions_Log.csv': useful: Meds - but not to be used for now. 

In [ ]:
### 'Current_Medical_Conditions_Log.csv': Meds
CURRCOND=dict_files[files_names[8]]
#CURRCOND.head()

### 'Adverse_Event_Log.csv': useful: Meds - but not to be used for now.
#### Measures the side effects of the meds taken for PD.

In [ ]:
LOG=dict_files[files_names[9]]
#LOG.head()

### 'Surgery_for_Parkinson_Disease.csv': useful: Meds - but not to be used for now.

In [ ]:
SURGPD=dict_files[files_names[12]]
#SURGPD.head()

## Datasets with two few data points (not enough patients)

### Skin_Biopsy - useful: Feat-bio (wound closure) - only 26 patients...

In [ ]:
SKBIO=dict_files[files_names[1]]
#SKBIO.head()
#SKBIO.columns

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT','SKBIOCMP','WOUNDCLS']
SKBIO_sel=SKBIO[SEL]
print(number_of_patients(SKBIO))
SKBIO_sel.head()
#list(set(SKBIO_sel['EVENT_ID']))

### IPSC_Labs: useful - Feat-bio: blood, liver disease, heparin(?), myeloma(?) stuff (linked to Med dose history)
### ONLY 137 patients...

In [ ]:
### ISPC_Labs: useful - Feat-bio
#### Select only history of liver disease and multiple myeloma (can be useful for treatments strategies)
IPSCLAB=dict_files[files_names[5]]
#IPSCLAB.head()
IPSCLAB.columns

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PDMEDYN', 'PDMEDDT', 'PDMEDTM', 'WARFBEF', 'HEPBEF', 'HXLIVER',
       'HXMMYLMA', 'BLDLTH', 'LTHTM', 'LTHINVN', 'BLDSST', 'BLDCPT']
# we might be able to remove some features. 
print(number_of_patients(IPSCLAB))
IPSCLAB_sel=IPSCLAB[SEL]
IPSCLAB_sel.head()
#list(set(IPSCLAB_sel['EVENT_ID']))

### 'Clinical_Diagnosis_and_Management.csv': Feat-PD - only 73 patients...

In [16]:
### 'Clinical_Diagnosis_and_Management.csv': Feat-PD
CLINDX=dict_files[files_names[2]]
CLINDX.head()
#CLINDX.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PSLVL', 'DCNOMTR', 'DCRTREM', 'DCRIGID', 'DCBRADY', 'DFPGDIST',
       'DFOTH', 'PRIMDIAG', 'OTHNEURO', 'CLDXCHNG', 'DXFDTIMG',
       'DXFCLSGN', 'DXFRSPMD', 'DXFNATHX', 'DXFOTH', 'DXFOTHCM', 'MNGCHNG',
       'MNGTRTSX', 'MNGTRIAL', 'MNGTRTOT', 'MNGDXTST', 'MNGNOTRT',
       'NEURWKDX']
print(number_of_patients(CLINDX))
CLINDX_sel=CLINDX[SEL]
CLINDX_sel.head()
#list(set(CLINDX_sel['EVENT_ID']))

(73, 11)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,PSLVL,DCNOMTR,DCRTREM,DCRIGID,DCBRADY,...,DXFNATHX,DXFOTH,DXFOTHCM,MNGCHNG,MNGTRTSX,MNGTRIAL,MNGTRTOT,MNGDXTST,MNGNOTRT,NEURWKDX
0,317677601,3408,BL,CLINDX,10/2011,3,0,0.0,1.0,1.0,...,1.0,0.0,NaN,0,0.0,0.0,0.0,0.0,1.0,NaN
1,317614301,3520,BL,CLINDX,10/2011,2,0,1.0,1.0,1.0,...,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,1.0,NaN
2,318556601,3426,BL,CLINDX,10/2011,3,0,1.0,0.0,1.0,...,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,1.0,Sciatica
3,318628801,3512,V02,CLINDX,10/2011,2,0,1.0,1.0,0.0,...,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,NaN
4,318729401,3425,BL,CLINDX,10/2011,2,0,0.0,1.0,1.0,...,0.0,0.0,NaN,0,0.0,0.0,0.0,1.0,0.0,spinal stenosis
